# EndoMamba 项目环境配置和依赖安装指南

本笔记本提供了在云服务器上设置 EndoMamba 项目完整环境的详细步骤，包括：

📋 **配置要求**
- Python 3.9
- CUDA 12.4  
- PyTorch 2.4.1+cu121
- 各种深度学习和计算机视觉库
- Mamba SSM 和 causal-conv1d 自定义构建

⚡ **主要功能**
1. 🔍 环境检测和验证
2. 📦 依赖包安装
3. 🔨 源代码构建
4. ✅ 功能验证测试

💡 **提示**: 请按顺序执行每个步骤，确保每个阶段都成功完成后再进行下一步。

## 1. 云服务器环境检测 🔍

在开始安装之前，我们需要检测当前系统环境，包括操作系统、Python版本、CUDA版本、PyTorch版本，以及硬件配置信息（GPU、CPU、内存）。

In [1]:
# 检测系统基本信息
import sys
import platform
import os
import subprocess
import psutil

print("=" * 60)
print("🖥️  系统基本信息")
print("=" * 60)
print(f"操作系统: {platform.system()} {platform.release()}")
print(f"架构: {platform.machine()}")
print(f"Python 版本: {sys.version}")
print(f"Python 可执行文件路径: {sys.executable}")

# 检测CPU信息
print("\n" + "=" * 60)
print("🔧 CPU 信息")
print("=" * 60)
print(f"CPU 核心数: {psutil.cpu_count(logical=False)} 物理核心")
print(f"CPU 线程数: {psutil.cpu_count(logical=True)} 逻辑核心")
cpu_freq = psutil.cpu_freq()
if cpu_freq:
    print(f"CPU 频率: {cpu_freq.current:.2f} MHz (最大: {cpu_freq.max:.2f} MHz)")

# 检测内存信息
print("\n" + "=" * 60)
print("💾 内存信息")
print("=" * 60)
memory = psutil.virtual_memory()
print(f"总内存: {memory.total / (1024**3):.2f} GB")
print(f"可用内存: {memory.available / (1024**3):.2f} GB")
print(f"已使用内存: {memory.used / (1024**3):.2f} GB")
print(f"内存使用率: {memory.percent:.1f}%")

🖥️  系统基本信息
操作系统: Linux 5.15.0-112-generic
架构: x86_64
Python 版本: 3.9.19 (main, May  6 2024, 19:43:03) 
[GCC 11.2.0]
Python 可执行文件路径: /root/miniconda/bin/python

🔧 CPU 信息
CPU 核心数: 36 物理核心
CPU 线程数: 72 逻辑核心
CPU 频率: 1379.17 MHz (最大: 3700.00 MHz)

💾 内存信息
总内存: 251.33 GB
可用内存: 241.47 GB
已使用内存: 8.08 GB
内存使用率: 3.9%


In [2]:
# 检测CUDA和GPU信息
print("\n" + "=" * 60)
print("🎮 GPU 和 CUDA 信息")
print("=" * 60)

# 检测NVIDIA GPU
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
    print("✅ NVIDIA GPU 检测成功:")
    lines = result.stdout.split('\n')
    for line in lines:
        if 'NVIDIA-SMI' in line:
            print(f"NVIDIA-SMI 版本: {line.strip()}")
        elif 'CUDA Version' in line:
            cuda_version = line.split('CUDA Version: ')[1].split()[0] if 'CUDA Version: ' in line else "未知"
            print(f"CUDA 驱动版本: {cuda_version}")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("❌ 未检测到 NVIDIA GPU 或 nvidia-smi 未安装")

# 检测PyTorch中的CUDA支持
try:
    import torch
    print(f"\n✅ PyTorch 已安装: {torch.__version__}")
    print(f"CUDA 可用: {'✅ 是' if torch.cuda.is_available() else '❌ 否'}")
    if torch.cuda.is_available():
        print(f"CUDA 版本 (PyTorch): {torch.version.cuda}")
        print(f"cuDNN 版本: {torch.backends.cudnn.version()}")
        print(f"GPU 数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_memory = torch.cuda.get_device_properties(i).total_memory / (1024**3)
            print(f"GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")
except ImportError:
    print("❌ PyTorch 未安装")

# 检测CUDA Toolkit
try:
    result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True, check=True)
    print(f"\n✅ CUDA Toolkit 已安装:")
    for line in result.stdout.split('\n'):
        if 'release' in line:
            print(f"CUDA Toolkit 版本: {line.strip()}")
            break
except (subprocess.CalledProcessError, FileNotFoundError):
    print("\n❌ CUDA Toolkit (nvcc) 未安装或不在 PATH 中")


🎮 GPU 和 CUDA 信息
✅ NVIDIA GPU 检测成功:
NVIDIA-SMI 版本: | NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |

✅ PyTorch 已安装: 2.4.1+cu121
CUDA 可用: ✅ 是
CUDA 版本 (PyTorch): 12.1
cuDNN 版本: 90100
GPU 数量: 1
GPU 0: NVIDIA GeForce RTX 3090 (23.7 GB)

✅ CUDA Toolkit 已安装:
CUDA Toolkit 版本: Cuda compilation tools, release 12.4, V12.4.131


## 2. 安装 Python 3.9 🐍

如果当前 Python 版本不是 3.9，我们需要安装 Python 3.9 并设置虚拟环境。

In [ ]:
# 检查Python版本并安装Python 3.9
import sys
import subprocess

current_version = sys.version_info
print(f"当前 Python 版本: {current_version.major}.{current_version.minor}.{current_version.micro}")

if current_version.major == 3 and current_version.minor == 9:
    print("✅ Python 3.9 已安装且为当前版本")
else:
    print("⚠️  需要安装 Python 3.9")
    print("\n执行以下命令安装 Python 3.9 (Ubuntu/Debian):")
    commands = [
        "sudo apt update",
        "sudo apt install -y software-properties-common",
        "sudo add-apt-repository -y ppa:deadsnakes/ppa",
        "sudo apt update", 
        "sudo apt install -y python3.9 python3.9-dev python3.9-venv python3.9-distutils",
        "sudo apt install -y python3-pip",
        "python3.9 -m pip install --upgrade pip"
    ]
    
    for cmd in commands:
        print(f"$ {cmd}")
    
    print("\n⚠️  请在终端中手动执行上述命令，然后重新启动此笔记本")

In [ ]:
# 创建和激活 Python 3.9 虚拟环境
import os
import subprocess

env_name = "endomamba_env"
env_path = os.path.join(os.getcwd(), env_name)

print(f"📁 创建虚拟环境: {env_name}")
print(f"📍 环境路径: {env_path}")

try:
    # 检查虚拟环境是否已存在
    if os.path.exists(env_path):
        print("✅ 虚拟环境已存在")
    else:
        print("🔄 正在创建虚拟环境...")
        result = subprocess.run([
            sys.executable, "-m", "venv", env_path
        ], capture_output=True, text=True, check=True)
        print("✅ 虚拟环境创建成功")
    
    # 提供激活命令
    if os.name == 'nt':  # Windows
        activate_cmd = f"{env_path}\\Scripts\\activate"
    else:  # Linux/Mac
        activate_cmd = f"source {env_path}/bin/activate"
    
    print(f"\n💡 激活虚拟环境命令:")
    print(f"$ {activate_cmd}")
    print(f"\n💡 在虚拟环境中运行 Jupyter:")
    print(f"$ {activate_cmd}")
    print(f"$ pip install jupyter")
    print(f"$ jupyter notebook")
    
except subprocess.CalledProcessError as e:
    print(f"❌ 创建虚拟环境失败: {e}")
    print("请确保 Python 3.9 已正确安装")

## 3. 安装 CUDA 12.4 ⚡

安装 CUDA 12.4 工具包，配置环境变量，验证 CUDA 安装和 GPU 可用性。

In [ ]:
# CUDA 12.4 安装命令
import subprocess
import os

print("🔽 CUDA 12.4 安装步骤 (Ubuntu/Linux)")
print("=" * 60)

# 检查当前CUDA版本
try:
    result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True, check=True)
    print("✅ 当前CUDA状态:")
    for line in result.stdout.split('\n'):
        if 'release' in line:
            print(f"已安装版本: {line.strip()}")
            break
except (subprocess.CalledProcessError, FileNotFoundError):
    print("❌ CUDA Toolkit 未安装")

print("\n📋 安装 CUDA 12.4 的命令:")
cuda_commands = [
    "# 1. 下载 CUDA 12.4 安装包",
    "wget https://developer.download.nvidia.com/compute/cuda/12.4.0/local_installers/cuda_12.4.0_550.54.14_linux.run",
    "",
    "# 2. 给安装包执行权限",
    "chmod +x cuda_12.4.0_550.54.14_linux.run",
    "",
    "# 3. 执行安装 (静默安装，跳过驱动)",
    "sudo sh cuda_12.4.0_550.54.14_linux.run --silent --toolkit",
    "",
    "# 4. 添加环境变量到 ~/.bashrc",
    'echo \'export PATH=/usr/local/cuda-12.4/bin:$PATH\' >> ~/.bashrc',
    'echo \'export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH\' >> ~/.bashrc',
    "",
    "# 5. 重新加载环境变量",
    "source ~/.bashrc",
    "",
    "# 6. 验证安装",
    "nvcc --version"
]

for cmd in cuda_commands:
    print(cmd)

print(f"\n⚠️  重要提示:")
print("1. 请确保已安装了 NVIDIA 驱动 (版本 >= 550.54.14)")
print("2. 如果驱动版本过低，请先更新驱动")
print("3. 安装完成后需要重启终端或重新加载环境变量")

## 4. 安装 PyTorch 2.4.1+cu121 🔥

使用 pip 安装 PyTorch 2.4.1+cu121 版本，包括 torchvision 和 torchaudio，验证 CUDA 支持。

In [ ]:
# 安装 PyTorch 2.4.1+cu121
import subprocess
import sys

print("🔥 安装 PyTorch 2.4.1+cu121")
print("=" * 60)

# PyTorch 安装命令
pytorch_install_cmd = [
    sys.executable, "-m", "pip", "install",
    "--extra-index-url", "https://download.pytorch.org/whl/cu121",
    "torch==2.4.1+cu121",
    "torchvision==0.19.1+cu121", 
    "torchaudio==2.4.1+cu121"
]

print("📦 正在安装 PyTorch, torchvision, torchaudio...")
print("命令:", " ".join(pytorch_install_cmd))

try:
    result = subprocess.run(pytorch_install_cmd, capture_output=True, text=True, check=True)
    print("✅ PyTorch 安装成功!")
    if result.stdout:
        print("安装日志:")
        print(result.stdout[-500:])  # 显示最后500个字符
except subprocess.CalledProcessError as e:
    print(f"❌ PyTorch 安装失败: {e}")
    print("错误信息:")
    print(e.stderr)
    
print("\n🔍 验证 PyTorch 安装...")

In [ ]:
# 验证 PyTorch 安装和 CUDA 支持
try:
    import torch
    import torchvision
    import torchaudio
    
    print("✅ PyTorch 导入成功!")
    print(f"PyTorch 版本: {torch.__version__}")
    print(f"TorchVision 版本: {torchvision.__version__}")
    print(f"TorchAudio 版本: {torchaudio.__version__}")
    
    print(f"\n🎮 CUDA 支持:")
    print(f"CUDA 可用: {'✅ 是' if torch.cuda.is_available() else '❌ 否'}")
    
    if torch.cuda.is_available():
        print(f"CUDA 版本: {torch.version.cuda}")
        print(f"cuDNN 版本: {torch.backends.cudnn.version()}")
        print(f"cuDNN 启用: {'✅ 是' if torch.backends.cudnn.enabled else '❌ 否'}")
        print(f"GPU 设备数量: {torch.cuda.device_count()}")
        
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            
        # 测试简单的CUDA操作
        print(f"\n🧪 CUDA 功能测试:")
        x = torch.randn(3, 3).cuda()
        y = torch.randn(3, 3).cuda()
        z = x + y
        print(f"CUDA 张量运算测试: {'✅ 通过' if z.is_cuda else '❌ 失败'}")
        
    else:
        print("⚠️  CUDA 不可用，请检查:")
        print("1. NVIDIA 驱动是否正确安装")
        print("2. CUDA Toolkit 是否安装")
        print("3. PyTorch 是否为 CUDA 版本")
        
except ImportError as e:
    print(f"❌ PyTorch 导入失败: {e}")
    print("请检查 PyTorch 是否正确安装")

## 5. 安装其他依赖项 📦

批量安装 requirements.txt 中的依赖项，包括深度学习框架、计算机视觉库、数据科学工具等。

In [ ]:
pip install -r requirements.txt

## 6. 从源代码构建安装 causal-conv1d 🔨

克隆 causal-conv1d 源代码仓库，编译并安装，处理可能的构建错误和依赖问题。

In [ ]:
pip install ./videomamba/causal-conv1d --no-build-isolation

In [ ]:
# 测试 causal-conv1d 基本功能 🧪
import torch
import numpy as np

print("🧪 测试 causal-conv1d 基本功能")
print("=" * 60)

try:
    from causal_conv1d import causal_conv1d_fn
    
    # 设置测试参数
    batch_size = 2
    seq_len = 128
    dim = 64
    width = 4
    
    print(f"📋 测试参数:")
    print(f"  批次大小: {batch_size}")
    print(f"  序列长度: {seq_len}")
    print(f"  特征维度: {dim}")
    print(f"  卷积宽度: {width}")
    
    # 创建测试数据
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🎮 使用设备: {device}")
    
    # 输入张量 (batch, dim, seq_len)
    x = torch.randn(batch_size, dim, seq_len, device=device, dtype=torch.float32)
    
    # 权重张量 (dim, width)
    weight = torch.randn(dim, width, device=device, dtype=torch.float32)
    
    # 偏置张量 (dim,)
    bias = torch.randn(dim, device=device, dtype=torch.float32)
    
    print(f"\n📊 张量形状:")
    print(f"  输入 x: {x.shape}")
    print(f"  权重 weight: {weight.shape}")
    print(f"  偏置 bias: {bias.shape}")
    
    # 执行 causal conv1d 操作
    print(f"\n🔄 执行 causal conv1d 运算...")
    
    # 测试不同的激活函数
    activations = ['silu', 'swish', None]
    
    for activation in activations:
        try:
            if activation:
                output = causal_conv1d_fn(x, weight, bias, activation=activation)
                print(f"✅ {activation} 激活函数测试通过，输出形状: {output.shape}")
            else:
                output = causal_conv1d_fn(x, weight, bias)
                print(f"✅ 无激活函数测试通过，输出形状: {output.shape}")
                
            # 验证输出的合理性
            if output.shape == x.shape:
                print(f"  ✓ 输出形状正确")
            else:
                print(f"  ❌ 输出形状不匹配，期望: {x.shape}, 实际: {output.shape}")
                
            # 检查输出是否包含有效数值
            if torch.isfinite(output).all():
                print(f"  ✓ 输出数值有效")
            else:
                print(f"  ❌ 输出包含无效数值 (NaN/Inf)")
                
        except Exception as e:
            print(f"❌ {activation or '无激活'} 测试失败: {e}")
    
    # 性能测试
    print(f"\n⚡ 性能测试:")
    
    # 预热
    for _ in range(5):
        _ = causal_conv1d_fn(x, weight, bias)
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # 计时
    import time
    start_time = time.time()
    num_iterations = 100
    
    for _ in range(num_iterations):
        output = causal_conv1d_fn(x, weight, bias)
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    end_time = time.time()
    avg_time = (end_time - start_time) / num_iterations * 1000  # 毫秒
    
    print(f"  平均执行时间: {avg_time:.2f} ms")
    print(f"  吞吐量: {batch_size * seq_len / (avg_time / 1000):.0f} tokens/sec")
    
    # 内存使用测试
    if device.type == 'cuda':
        memory_before = torch.cuda.memory_allocated(device) / 1024**2
        
        # 创建较大的张量进行测试
        large_x = torch.randn(8, 512, 2048, device=device, dtype=torch.float32)
        large_weight = torch.randn(512, 4, device=device, dtype=torch.float32)
        large_bias = torch.randn(512, device=device, dtype=torch.float32)
        
        large_output = causal_conv1d_fn(large_x, large_weight, large_bias)
        
        memory_after = torch.cuda.memory_allocated(device) / 1024**2
        memory_used = memory_after - memory_before
        
        print(f"\n💾 内存使用测试:")
        print(f"  测试张量大小: {large_x.shape}")
        print(f"  内存使用量: {memory_used:.1f} MB")
        
        # 清理内存
        del large_x, large_weight, large_bias, large_output
        torch.cuda.empty_cache()
    
    print(f"\n🎉 causal-conv1d 功能测试全部通过!")
    print(f"✅ 库安装正确，可以正常使用")
    
except Exception as e:
    print(f"❌ causal-conv1d 功能测试失败: {e}")
    print(f"💡 可能的问题:")
    print(f"1. CUDA 环境配置不正确")
    print(f"2. PyTorch 版本不兼容")
    print(f"3. 编译时出现问题")
    
    # 提供调试信息
    print(f"\n🔍 调试信息:")
    try:
        import causal_conv1d
        print(f"causal_conv1d 版本: {getattr(causal_conv1d, '__version__', '未知')}")
        print(f"causal_conv1d 路径: {causal_conv1d.__file__}")
    except:
        print("无法获取 causal_conv1d 调试信息")

print("\n" + "=" * 60)

## 7. 从源代码构建安装自定义 Mamba 🐍

下载并编译安装 Mamba SSM 库，配置自定义版本，确保与项目兼容。

In [ ]:
cd ./videomamba/_mamba
export MAMBA_FORCE_BUILD=TRUE
python setup.py clean
rm -rf build/ *.egg-info
pip install . --no-build-isolation --verbose

## 8. 安装验证和快速演示测试 ✅

运行 endomamba_demo.py 演示脚本，验证所有组件正常工作，测试模型加载和推理功能。

In [2]:
# 完整环境验证
import sys
import importlib

print("🔍 完整环境验证")
print("=" * 60)

# 关键库验证列表
key_libraries = [
    ("torch", "PyTorch"),
    ("torchvision", "TorchVision"), 
    ("torchaudio", "TorchAudio"),
    ("numpy", "NumPy"),
    ("cv2", "OpenCV"),
    ("PIL", "Pillow"),
    ("timm", "Timm"),
    ("einops", "Einops"),
    ("transformers", "Transformers"),
    ("wandb", "Weights & Biases"),
    ("sklearn", "Scikit-learn"),
    ("pandas", "Pandas"),
    ("matplotlib", "Matplotlib"),
    ("tqdm", "TQDM"),
    ("yaml", "PyYAML"),
    ("causal_conv1d", "Causal Conv1D"),
    ("mamba_ssm", "Mamba SSM"),
]

print("📦 验证关键库安装状态:")
print("-" * 40)

success_count = 0
total_count = len(key_libraries)

for module_name, display_name in key_libraries:
    try:
        module = importlib.import_module(module_name)
        version = getattr(module, '__version__', '未知版本')
        print(f"✅ {display_name}: {version}")
        success_count += 1
    except ImportError:
        print(f"❌ {display_name}: 未安装")

print(f"\n📊 安装统计: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")

# CUDA 功能测试
print(f"\n🎮 CUDA 功能测试:")
print("-" * 40)
try:
    import torch
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(f"✅ CUDA 设备: {torch.cuda.get_device_name()}")
        
        # 创建测试张量并进行计算
        x = torch.randn(1000, 1000, device=device)
        y = torch.randn(1000, 1000, device=device)
        z = torch.mm(x, y)
        print(f"✅ CUDA 矩阵运算测试通过")
        
        # 测试内存
        memory_allocated = torch.cuda.memory_allocated(device) / 1024**2
        memory_cached = torch.cuda.memory_reserved(device) / 1024**2
        print(f"📊 GPU 内存使用: {memory_allocated:.1f}MB 已分配, {memory_cached:.1f}MB 已缓存")
        
    else:
        print("❌ CUDA 不可用")
except Exception as e:
    print(f"❌ CUDA 测试失败: {e}")

if success_count >= total_count * 0.8:  # 80%成功率
    print(f"\n🎉 环境验证通过! 可以继续演示测试")
else:
    print(f"\n⚠️  环境验证不完整，请检查失败的库安装")

🔍 完整环境验证
📦 验证关键库安装状态:
----------------------------------------
❌ PyTorch: 未安装
❌ TorchVision: 未安装
❌ TorchAudio: 未安装
❌ NumPy: 未安装
❌ OpenCV: 未安装
❌ Pillow: 未安装
❌ Timm: 未安装
❌ Einops: 未安装
❌ Transformers: 未安装
❌ Weights & Biases: 未安装
❌ Scikit-learn: 未安装
❌ Pandas: 未安装
❌ Matplotlib: 未安装
✅ TQDM: 4.66.2
✅ PyYAML: 6.0.2
❌ Causal Conv1D: 未安装
❌ Mamba SSM: 未安装

📊 安装统计: 2/17 (11.8%)

🎮 CUDA 功能测试:
----------------------------------------
❌ CUDA 测试失败: No module named 'torch'

⚠️  环境验证不完整，请检查失败的库安装


In [ ]:
# 运行 EndoMamba 演示脚本
import subprocess
import sys
import os

print("🚀 运行 EndoMamba 演示测试")
print("=" * 60)

# 检查演示脚本路径
demo_script_path = "videomamba/tests/endomamba_demo.py"

if os.path.exists(demo_script_path):
    print(f"✅ 找到演示脚本: {demo_script_path}")
    
    try:
        print("🔄 正在运行演示脚本...")
        print("⚠️  注意: 首次运行可能需要下载预训练模型，请耐心等待")
        
        # 切换到正确的目录
        original_dir = os.getcwd()
        test_dir = "videomamba/tests"
        
        os.chdir(test_dir)
        print(f"📁 当前目录: {os.getcwd()}")
        
        # 运行演示脚本
        result = subprocess.run([
            sys.executable, "endomamba_demo.py"
        ], capture_output=True, text=True, timeout=300, check=True)  # 5分钟超时
        
        print("✅ 演示脚本执行成功!")
        print("\n📋 输出结果:")
        print("-" * 40)
        print(result.stdout)
        
        if result.stderr:
            print("\n⚠️  警告信息:")
            print(result.stderr)
            
        os.chdir(original_dir)
        
    except subprocess.TimeoutExpired:
        print("⏰ 演示脚本运行超时 (5分钟)")
        print("💡 这可能是由于模型下载或计算时间过长")
        os.chdir(original_dir)
        
    except subprocess.CalledProcessError as e:
        print(f"❌ 演示脚本执行失败: {e}")
        print("\n错误输出:")
        print(e.stderr)
        print("\n💡 可能的问题:")
        print("1. 缺少预训练模型文件")
        print("2. GPU 内存不足")
        print("3. 某些依赖库未正确安装")
        os.chdir(original_dir)
        
    except Exception as e:
        print(f"❌ 运行演示时发生错误: {e}")
        os.chdir(original_dir)
        
else:
    print(f"❌ 未找到演示脚本: {demo_script_path}")
    print("💡 请确保在 EndoMamba 项目根目录运行此脚本")

## 🎉 安装完成总结

恭喜！您已经完成了 EndoMamba 项目的完整环境配置。

### ✅ 已完成的步骤:

1. **环境检测** - 检查系统配置和硬件信息
2. **Python 3.9** - 安装和配置 Python 3.9 环境  
3. **CUDA 12.4** - 安装 CUDA 工具包和驱动
4. **PyTorch 2.4.1+cu121** - 安装 GPU 支持的 PyTorch
5. **依赖项安装** - 批量安装所有必需的 Python 库
6. **Causal-Conv1D** - 从源代码构建自定义卷积库
7. **Mamba SSM** - 安装状态空间模型库
8. **功能验证** - 运行演示脚本验证系统

### 🚀 后续使用指南:

#### 训练模型
```bash
cd videomamba/video_sm
python run_endomamba_pretraining.py --config configs/your_config.yaml
```

#### 微调模型  
```bash
python run_class_finetuning.py --model endomamba_small --dataset your_dataset
```

#### 推理预测
```bash
python run_inference.py --model_path /path/to/model --input_video /path/to/video
```

### 📚 重要文档:
- 项目文档: `README.md`
- 数据集配置: `videomamba/video_sm/DATASET.md`
- 模型配置: `videomamba/video_sm/models/`

### ⚠️  常见问题:
1. **GPU 内存不足**: 减少 batch_size 或使用梯度累积
2. **模型下载慢**: 使用国内镜像或手动下载
3. **编译错误**: 确保 CUDA 和 gcc 版本兼容

### 📞 技术支持:
如果遇到问题，请检查:
- CUDA 和 PyTorch 版本兼容性
- GPU 驱动是否最新
- 所有依赖项是否正确安装

**🎊 环境配置完成，开始您的 EndoMamba 之旅！**